<a href="https://colab.research.google.com/github/EdonFetaji/DataScience/blob/main/labs/lab4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import yfinance
from datetime import datetime


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM,Input,Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [81]:
df = yfinance.download('BTC-USD', start='2009-01-01', end='2024-12-26', interval='1d')
print(df.info())


[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3753 entries, 2014-09-17 to 2024-12-25
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (Adj Close, BTC-USD)  3753 non-null   float64
 1   (Close, BTC-USD)      3753 non-null   float64
 2   (High, BTC-USD)       3753 non-null   float64
 3   (Low, BTC-USD)        3753 non-null   float64
 4   (Open, BTC-USD)       3753 non-null   float64
 5   (Volume, BTC-USD)     3753 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 205.2 KB
None


In [82]:
df = df[['Close','Volume']]
df

Price,Close,Volume
Ticker,BTC-USD,BTC-USD
Date,,
2014-09-17,457.334015,21056800
2014-09-18,424.440002,34483200
2014-09-19,394.795990,37919700
2014-09-20,408.903992,36863600
2014-09-21,398.821014,26580100
...,...,...
2024-12-21,97224.726562,51765334294
2024-12-22,95104.937500,43147981314


In [ ]:
sentiment = pd.read_csv('/content/bitcoin_articles.csv')
sentiment

In [87]:
sentiment['published_date'] = sentiment['published_date'].apply(lambda x:  datetime.strptime(x, "%d-%m-%Y %H:%M").date())

<ipython-input-87-2bb40b94acde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment['published_date'] = sentiment['published_date'].apply(lambda x:  datetime.strptime(x, "%d-%m-%Y %H:%M").date())


In [85]:
sentiment = sentiment [['published_date','summary']]

In [74]:
sentiment

,published_date,summary
0,2022-10-04,"Over the course of the last few years, there h..."
1,2022-10-05,Russia Legalizing Bitcoin And Crypto Is A Matt...
2,2022-10-04,Bitcoin has a maximum supply of 21 million. Ho...
3,2022-10-03,"Conceived in 2008 and launched in 2009, bitcoi..."
4,2022-10-03,Bear markets are for building out capacity on ...
...,...,...
2495,2022-10-03,New research published by an international com...
2496,2022-10-04,"Crypto was the best performing asset in Q3, 20..."
2497,2022-10-04,I wish everyone a good day.\nI sell footer bac...
2498,2022-10-05,The chairman of the U.S. Securities and Exchan...


In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline


In [75]:
model_name = "yiyanghkust/finbert-tone"
sentiment_analyzer = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)


Device set to use cuda:0


In [86]:
sentiment.isna().sum()

,0
published_date,0
summary,27


In [88]:
sentiment.dropna(subset="summary",axis=0,inplace=True)

<ipython-input-88-e361c7c28331>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment.dropna(subset="summary",axis=0,inplace=True)


In [89]:
sentiment['sentiment'] = sentiment['summary'].apply(lambda x: sentiment_analyzer(x)[0]['label'])

<ipython-input-89-cd1742047d2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment['sentiment'] = sentiment['summary'].apply(lambda x: sentiment_analyzer(x)[0]['label'])


In [92]:
sentiment = sentiment.sort_values('published_date')
df = df.sort_values('Date')

In [26]:
df = df.reset_index()

In [91]:
df.sample(10)

Price,Date,Close,Volume
Ticker,,BTC-USD,BTC-USD
2920,2022-09-15,19701.210938,36389011503
1645,2019-03-20,4087.476318,10175916388
259,2015-06-03,225.873993,17752400
2839,2022-06-26,21027.294922,18027170497
3627,2024-08-22,60381.914062,27625734377
2032,2020-04-10,6865.493164,43622840992
3287,2023-09-17,26534.187500,6774210670
1211,2018-01-10,14973.299805,18500800512
2745,2022-03-24,43960.933594,31042992291


In [93]:
sentiment

,published_date,summary,sentiment
2499,2022-10-03,Cryptocurrencies are a digital-only form of fi...,Neutral
1464,2022-10-03,53 minutes ago\nCrypto World News\nBitcoin Mag...,Neutral
1459,2022-10-03,Robert Kiyosaki Urges to Invest in ‘Real Money...,Neutral
1458,2022-10-03,Bitcoin mining GMT Token unveils their new NFT...,Positive
487,2022-10-03,Bitcoin balances and revenues hit all-time hig...,Positive
...,...,...,...
188,2022-10-09,According to countdown statistics based on the...,Neutral
1154,2022-10-09,Bitcoin Metaverse (BITMETA) traded up 6% again...,Neutral
2316,2022-10-09,3 hours ago\nFlash News\nBeInCrypto News\n 236...,Positive
60,2022-10-09,Crypto mining companies have struggled mightil...,Positive


In [90]:
df = df.reset_index()

In [94]:
sentiment['published_date'] = pd.to_datetime(sentiment['published_date'])
df['Date'] = pd.to_datetime(df['Date'])

In [95]:
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

df.columns

Index(['Date', 'Close', 'Volume'], dtype='object')

In [96]:
merged_df = pd.merge(sentiment, df, left_on='published_date',right_on='Date')
merged_df

,published_date,summary,sentiment,Date,Close,Volume
0,2022-10-03,Cryptocurrencies are a digital-only form of fi...,Neutral,2022-10-03,19623.580078,30484729489
1,2022-10-03,53 minutes ago\nCrypto World News\nBitcoin Mag...,Neutral,2022-10-03,19623.580078,30484729489
2,2022-10-03,Robert Kiyosaki Urges to Invest in ‘Real Money...,Neutral,2022-10-03,19623.580078,30484729489
3,2022-10-03,Bitcoin mining GMT Token unveils their new NFT...,Positive,2022-10-03,19623.580078,30484729489
4,2022-10-03,Bitcoin balances and revenues hit all-time hig...,Positive,2022-10-03,19623.580078,30484729489
...,...,...,...,...,...,...
2468,2022-10-09,According to countdown statistics based on the...,Neutral,2022-10-09,19446.425781,16837262532
2469,2022-10-09,Bitcoin Metaverse (BITMETA) traded up 6% again...,Neutral,2022-10-09,19446.425781,16837262532
2470,2022-10-09,3 hours ago\nFlash News\nBeInCrypto News\n 236...,Positive,2022-10-09,19446.425781,16837262532
2471,2022-10-09,Crypto mining companies have struggled mightil...,Positive,2022-10-09,19446.425781,16837262532


In [97]:
merged_df.drop(columns=['summary','published_date','Date'],inplace=True,axis=1)
merged_df

,sentiment,Close,Volume
0,Neutral,19623.580078,30484729489
1,Neutral,19623.580078,30484729489
2,Neutral,19623.580078,30484729489
3,Positive,19623.580078,30484729489
4,Positive,19623.580078,30484729489
...,...,...,...
2468,Neutral,19446.425781,16837262532
2469,Neutral,19446.425781,16837262532
2470,Positive,19446.425781,16837262532
2471,Positive,19446.425781,16837262532


In [ ]:
merged_df['sentiment'] = merged_df.sentiment.map({
    'Positive':1,
    'Neutral': 0,
    'Negative' : -1
})
merged_df

In [101]:
merged_df.columns

Index(['sentiment', 'Close', 'Volume'], dtype='object')

In [102]:
copy = merged_df.copy()
for i in range(7, -1, -1):
    for col in merged_df.columns:
        copy[f"{col}_lag_{i}"] = merged_df[col].shift(i)


In [103]:
df = copy.dropna()
df

,sentiment,Close,Volume,sentiment_lag_7,Close_lag_7,Volume_lag_7,sentiment_lag_6,Close_lag_6,Volume_lag_6,sentiment_lag_5,...,Volume_lag_3,sentiment_lag_2,Close_lag_2,Volume_lag_2,sentiment_lag_1,Close_lag_1,Volume_lag_1,sentiment_lag_0,Close_lag_0,Volume_lag_0
7,1,19623.580078,30484729489,0.0,19623.580078,3.048473e+10,0.0,19623.580078,3.048473e+10,0.0,...,3.048473e+10,0.0,19623.580078,3.048473e+10,0.0,19623.580078,3.048473e+10,1,19623.580078,30484729489
8,1,19623.580078,30484729489,0.0,19623.580078,3.048473e+10,0.0,19623.580078,3.048473e+10,1.0,...,3.048473e+10,0.0,19623.580078,3.048473e+10,1.0,19623.580078,3.048473e+10,1,19623.580078,30484729489
9,0,19623.580078,30484729489,0.0,19623.580078,3.048473e+10,1.0,19623.580078,3.048473e+10,1.0,...,3.048473e+10,1.0,19623.580078,3.048473e+10,1.0,19623.580078,3.048473e+10,0,19623.580078,30484729489
10,0,19623.580078,30484729489,1.0,19623.580078,3.048473e+10,1.0,19623.580078,3.048473e+10,0.0,...,3.048473e+10,1.0,19623.580078,3.048473e+10,0.0,19623.580078,3.048473e+10,0,19623.580078,30484729489
11,0,19623.580078,30484729489,1.0,19623.580078,3.048473e+10,0.0,19623.580078,3.048473e+10,0.0,...,3.048473e+10,0.0,19623.580078,3.048473e+10,0.0,19623.580078,3.048473e+10,0,19623.580078,30484729489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,0,19446.425781,16837262532,0.0,19446.425781,1.683726e+10,0.0,19446.425781,1.683726e+10,1.0,...,1.683726e+10,1.0,19446.425781,1.683726e+10,0.0,19446.425781,1.683726e+10,0,19446.425781,16837262532
2469,0,19446.425781,16837262532,0.0,19446.425781,1.683726e+10,1.0,19446.425781,1.683726e+10,0.0,...,1.683726e+10,0.0,19446.425781,1.683726e+10,0.0,19446.425781,1.683726e+10,0,19446.425781,16837262532
2470,1,19446.425781,16837262532,1.0,19446.425781,1.683726e+10,0.0,19446.425781,1.683726e+10,1.0,...,1.683726e+10,0.0,19446.425781,1.683726e+10,0.0,19446.425781,1.683726e+10,1,19446.425781,16837262532
2471,1,19446.425781,16837262532,0.0,19446.425781,1.683726e+10,1.0,19446.425781,1.683726e+10,1.0,...,1.683726e+10,0.0,19446.425781,1.683726e+10,1.0,19446.425781,1.683726e+10,1,19446.425781,16837262532


In [105]:
x = df.drop(columns=['Close'],axis=1)
y = df['Close']

In [106]:
train_x,test_x,train_y,test_y=  train_test_split(x,y,test_size=0.2,shuffle=False)

In [107]:
scaler = MinMaxScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [109]:
X_train_lstm = np.expand_dims(train_x, axis=1)
X_test_lstm = np.expand_dims(test_x, axis=1)

In [110]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dropout(0.2),
    Dense(1)
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [111]:
model.compile(loss='mse',optimizer='adam',metrics=['mae'])

In [112]:
history = model.fit(X_train_lstm,train_y,validation_split=0.2, epochs=69, batch_size=8)

Epoch 1/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 401493888.0000 - mae: 20035.2891 - val_loss: 390135072.0000 - val_mae: 19751.0566
Epoch 2/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 398007072.0000 - mae: 19948.5020 - val_loss: 385258944.0000 - val_mae: 19627.6855
Epoch 3/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 390582720.0000 - mae: 19762.1016 - val_loss: 377682816.0000 - val_mae: 19434.0156
Epoch 4/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 380079008.0000 - mae: 19495.0293 - val_loss: 367310912.0000 - val_mae: 19164.9258
Epoch 5/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 366052416.0000 - mae: 19131.7363 - val_loss: 354751552.0000 - val_mae: 18832.6602
Epoch 6/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 349237792.0000 - mae: 18685.6387 - val_loss: 340505152.0000 - val_mae: 18446.6895
Epoch 7/69
198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 330824192.0000 - mae: 18182.5352 - val_loss: 324815552.0000 - val_mae: 18009.5410
Epoch

In [113]:
def evaluate_prediction(orig_y,pred_y):
  print(f"MSE: ",mean_squared_error(orig_y,pred_y))
  print(f"MAE: ",mean_absolute_error(orig_y,pred_y))
  print(f"R2 SCORE: ",r2_score(orig_y,pred_y))

In [114]:
pred_y = model.predict(X_test_lstm)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step


In [115]:
evaluate_prediction(test_y,pred_y)

MSE:  275601083.5767132
MAE:  15802.106940848626
R2 SCORE:  -87640.5606165602
